In [1]:
import pandas as pd
import json
import minsearch

# Ingestion

In [139]:
with open('../data/arsonor_chunks_300_50.json', 'r', encoding='utf-8') as file:
    documents = json.load(file)

In [140]:
documents[15]

{'article_id': '3632a3b4',
 'url': 'https://arsonor.com/lintelligence-artificielle-ia-dans-le-studio-de-production-audio-5-6/',
 'title': 'L’intelligence artificielle (IA) dans le studio de production audio (5/6)',
 'category': 'LA POST-PROD',
 'tags': 'dé-mixage, de-noise, de-reverb, deep learning, plug-in audio, restauration audio, stems',
 'chunk_id': '3632a3b4-11',
 'chunk_text': "Ce plugin, de part sa simplicité d’utilisation (un ou deux potards à régler) est vite devenu une référence absolue, un « must-have » auprès de tout les ingénieurs du son d’un studio de musique. Plug-in Drumatom de Accusonus Accusonus Regroover pour séparer les éléments d'une boucle de batterie Un autre de leurs logiciels remarquables est Regroover . Celui-ci permet de décomposer efficacement une boucle audio complexe (souvent rythmique, de batterie) en plusieurs boucles contenant chacune un élément/instrument séparé de la boucle principale! Très utile par exemple pour isoler le kick, la snare, les high ha

### With Minsearch:

In [73]:
index = minsearch.Index(
    text_fields=['title', 'tags', 'chunk_text'],
    keyword_fields=['article_id', 'category', 'chunk_id']
)

In [74]:
index.fit(documents)

### With Elasticsearch

In [6]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import json

In [7]:
es = Elasticsearch("http://localhost:9200")

In [142]:
index_name = "arsonor_chunks_300"

# Create index if not already created
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body={
        "mappings": {
            "properties": {
                "article_id": {"type": "keyword"},
                "title": {"type": "text"},
                "url": {"type": "keyword"},
                "category": {"type": "keyword"},
                "tags": {"type": "text"},
                "chunk_id": {"type": "keyword"},
                "chunk_text": {"type": "text"},
            }
        }
    })

In [143]:
def prepare_documents_for_indexing(docs):
    for doc in docs:
        yield {
            "_index": index_name,
            "_id": doc['chunk_id'],
            "_source": {
                "article_id": doc['article_id'],
                "title": doc['title'],
                "url": doc['url'],
                "category": doc['category'],
                "tags": doc['tags'],
                "chunk_id": doc['chunk_id'],
                "chunk_text": doc['chunk_text'],
            }
        }

# Index the documents in bulk
bulk(es, prepare_documents_for_indexing(documents))

(572, [])

# RAG flow

In [110]:
from openai import OpenAI
client = OpenAI()

In [111]:
query = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'

In [12]:
response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": query}]
    )
    
response.choices[0].message.content

"Pour créer de la musique dans un home-studio, voici une liste du matériel nécessaire :\n\n### 1. **Ordinateur**\n   - Un ordinateur performant (PC ou Mac) avec une bonne quantité de RAM et un processeur rapide pour faire tourner des logiciels de production musicale.\n\n### 2. **Logiciel de Production Musicale (DAW)**\n   - Logiciels tels que Ableton Live, FL Studio, Logic Pro, Pro Tools, ou Reaper pour l'enregistrement, l'édition et la production de musique.\n\n### 3. **Interfaces Audio**\n   - Une interface audio pour connecter vos instruments et microphones à votre ordinateur avec une meilleure qualité sonore.\n\n### 4. **Moniteurs de Studio**\n   - Des enceintes de monitoring pour écouter votre musique avec précision. Choisissez des modèles adaptés à la taille de votre pièce.\n\n### 5. **Casque Audio**\n   - Un bon casque de studio pour le mixage et l'enregistrement, offrant une isolation phonique et une réponse plate.\n\n### 6. **Clavier MIDI**\n   - Un clavier MIDI pour jouer des

### Search results with Minsearch

In [75]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

### Search results with Elasticsearch

In [144]:
def elastic_search(query, category=None):
    search_query = {
        "size": 10,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["title", "tags", "chunk_text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    if category is not None:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "category": category
            }
        }

    response = es.search(index=index_name, body=search_query)
    
    return [hit['_source'] for hit in response['hits']['hits']]

### Prompt

In [112]:
prompt_template = """
You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally, recommend the top 3 Arsonor articles (refer to their 'title') that are the best to read for answering this question.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
article title: {title}
article keywords: {tags}
content: {chunk_text}
""".strip()


In [113]:
def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context += entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [114]:
search_results = elastic_search(query, category='LE HOME STUDIO')
prompt = build_prompt(query, search_results)

In [115]:
print(prompt)

You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally, recommend the top 3 Arsonor articles (refer to their 'title') that are the best to read for answering this question.

QUESTION: De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?

CONTEXT:
article title: Deviens toi aussi producteur musical depuis ton home studio
article keywords: home-studio, ingénieur du son, MAO, production musicale
content: Avis à tous les passionnés de musique, c’est avec grand plaisir que j’ouvre ce blog afin de partager ma passion de la MAO (Musique Assistée par Ordinateur) et du home studio , de la création au mixage audio. Alors tu te demandes sûrement… 

In [116]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [122]:
def rag(query, category=None, model='gpt-4o-mini'):
    search_results = elastic_search(query, category)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, model=model)
    return answer

In [124]:
# category = 'LE HOME STUDIO'
query = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'
answer = rag(query)
print(answer)

Pour créer de la musique dans votre home-studio, vous aurez besoin de plusieurs éléments essentiels :

1. **Un ordinateur** : C'est la base de votre home-studio. Un ordinateur relativement récent (Mac ou PC) suffira pour commencer la production musicale.
   
2. **Une interface audio** (optionnelle mais recommandée si vous souhaitez enregistrer des sons extérieurs) : Elle permet de faire entrer et sortir l'audio de votre ordinateur, facilitant ainsi l'enregistrement et la manipulation des sons.

3. **Un casque** ou des **moniteurs de studio** : Ils sont nécessaires pour écouter le son avec précision et effectuer le mixage de manière adéquate.

4. **Un logiciel de production musicale (DAW)** : C'est le cœur de votre home-studio, où vous allez enregistrer, éditer, mixer, et produire votre musique. 

5. **Plugins et instruments virtuels** : Ils vous permettent d'accéder à des sons et effets variés pour vos compositions.

En résumé, même si l'équipement peut sembler intimidant, un ordinateu

# Retrieval evaluation

In [145]:
df_question = pd.read_csv('../data/ground-truth-300.csv')
df_question.head()

,question,category,chunk,article
0,Quel est l'impact de l'IA sur la post-producti...,LA POST-PROD,4615db39-1,4615db39
1,Comment les outils IA simplifient-ils le trava...,LA POST-PROD,4615db39-1,4615db39
2,Quels avantages l'IA apporte-t-elle aux artist...,LA POST-PROD,4615db39-1,4615db39
3,Comment un débutant peut-il améliorer ses prod...,LA POST-PROD,4615db39-1,4615db39
4,Quelle est l'évolution des outils audio pour l...,LA POST-PROD,4615db39-1,4615db39


In [146]:
ground_truth = df_question.to_dict(orient='records')
ground_truth[0]

{'question': "Quel est l'impact de l'IA sur la post-production audio et musicale",
 'category': 'LA POST-PROD',
 'chunk': '4615db39-1',
 'article': '4615db39'}

In [26]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

### 1) Basic chunking, Minsearch based on the chunk_id

In [78]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [79]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['chunk']
        results = search_function(q)
        relevance = [d['chunk_id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [15]:
from tqdm.auto import tqdm

In [29]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2945 [00:00<?, ?it/s]

{'hit_rate': 0.5297113752122241, 'mrr': 0.3079710027757569}

### 2) Basic chunking, Minsearch based on the article_id

In [39]:
def evaluate2(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['article']
        results = search_function(q)
        
        # Ensure that only the first relevant result counts for the MRR
        relevance = []
        seen_article = False
        for d in results:
            if d['article_id'] == doc_id and not seen_article:
                relevance.append(True)
                seen_article = True
            else:
                relevance.append(False)

        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


In [40]:
evaluate2(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2945 [00:00<?, ?it/s]

{'hit_rate': 0.5911714770797962, 'mrr': 0.4871826070552721}

### 3) Find the best boost parameters

In [83]:
df_validation = df_question[:100]
df_test = df_question[100:]

In [58]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [59]:
gt_val = df_validation.to_dict(orient='records')

In [84]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [85]:
param_ranges = {
    'title': (0.0, 3.0),
    'tags': (0.0, 3.0),
    'chunk_text': (0.0, 3.0)
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [86]:
simple_optimize(param_ranges, objective, n_iterations=20)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

({'title': 0.5164183119720305,
  'tags': 1.172271157905152,
  'chunk_text': 2.2336086621388977},
 0.18979365079365076)

In [89]:
def minsearch_improved(query):
    boost = {
        'title': 0.31,
        'tags': 0.22,
        'chunk_text': 1.56
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [45]:
evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/2945 [00:00<?, ?it/s]

{'hit_rate': 0.664855687606112, 'mrr': 0.40281981836311226}

### 4) Dynamic chunking size 300, overlap 50 - Minsearch, text search

In [56]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2860 [00:00<?, ?it/s]

{'hit_rate': 0.529020979020979, 'mrr': 0.2984319846819851}

In [70]:
evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/2860 [00:00<?, ?it/s]

{'hit_rate': 0.848951048951049, 'mrr': 0.5353556166056164}

### 5) Dynamic chunking size 350, overlap 30 - Minsearch, text search

In [80]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2215 [00:00<?, ?it/s]

{'hit_rate': 0.5544018058690745, 'mrr': 0.31290193127664956}

In [90]:
evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/2215 [00:00<?, ?it/s]

{'hit_rate': 0.7972911963882618, 'mrr': 0.4890395571321085}

### 6) Dynamic chunking - Elasticsearch, text search

chunk size 300, overlap 50:

In [16]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['category']))

  0%|          | 0/2860 [00:00<?, ?it/s]

{'hit_rate': 0.8744755244755245, 'mrr': 0.607808719058718}

chunk size 300, overlap 50, boosted:

In [ ]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['category']))

chunk size 350, overlap 30:

In [94]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['category']))

  0%|          | 0/2215 [00:00<?, ?it/s]

{'hit_rate': 0.8600451467268623, 'mrr': 0.5898710093518226}

chunk size 350, overlap 30, boosted:

# RAG evaluation

In [98]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [147]:
len(ground_truth)

2860

In [148]:
record = ground_truth[0]
record

{'question': "Quel est l'impact de l'IA sur la post-production audio et musicale",
 'category': 'LA POST-PROD',
 'chunk': '4615db39-1',
 'article': '4615db39'}

In [149]:
question = record['question']
answer_llm = rag(question)

In [150]:
print(answer_llm)

L'impact de l'intelligence artificielle (IA) sur la post-production audio et musicale est significatif. Les outils et plug-ins basés sur l'IA révolutionnent le domaine, permettant à une plus grande variété d'utilisateurs—des amateurs aux professionnels—d'atteindre des résultats audio de qualité professionnelle sans nécessiter de formation approfondie en ingénierie du son. Cela favorise l'autonomie des artistes dans leur processus créatif, réduisant la dépendance à des experts pour des tâches courantes en post-production.

Les technologies IA facilitent la gestion de problèmes complexes dans le mixage et le mastering, rendant même le démarrage plus accessible pour les novices. Par exemple, les plug-ins utilisant l'IA peuvent automatiser des tâches comme l'harmonisation de samples audio ou la réduction du bruit, ce qui est une avancée majeure pour la restauration audio.

En résumé, l'IA continue de démocratiser l'accès à des outils de qualité en audio, tout en augmentant l'efficacité des

In [151]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: Quel est l'impact de l'IA sur la post-production audio et musicale
Generated Answer: L'impact de l'intelligence artificielle (IA) sur la post-production audio et musicale est significatif. Les outils et plug-ins basés sur l'IA révolutionnent le domaine, permettant à une plus grande variété d'utilisateurs—des amateurs aux professionnels—d'atteindre des résultats audio de qualité professionnelle sans nécessiter de formation approfondie en ingénierie du son. Cela favorise l'autonomie des artistes dans leur processus créatif, réduisant la dépendance à des experts pour des tâches courantes en post-production.

Les technologies IA facilitent la gestion de problèmes complexes dans le mixage et le mas

In [152]:
df_sample = df_question.sample(n=200, random_state=1)

In [153]:
sample = df_sample.to_dict(orient='records')

In [154]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [156]:
evaluations[0]

({'question': 'Pourquoi est-il nécessaire de faire des choix dans les zones fréquentielles pour chaque instrument',
  'category': 'LA POST-PROD',
  'chunk': '055c339f-6',
  'article': '055c339f'},
 "Faire des choix dans les zones fréquentielles pour chaque instrument est essentiel pour garantir un mixage clair et équilibré. Chaque instrument a une plage de fréquences spécifiques qui lui est associée, et lorsque plusieurs instruments sont joués ensemble, leur spectre peut se chevaucher. Cela peut conduire à un phénomène de masquage fréquentiel, où un instrument peut rendre difficile l'audition d'un autre parce que les fréquences qu'ils occupent se chevauchent. En se concentrant sur les bonnes zones fréquentielles pour chaque instrument, on peut créer un espace distinct pour chacun dans le mix, ce qui permet à chaque élément de se distinguer clairement et d'apporter sa contribution individuelle à l'ensemble.\n\nPour illustrer, par exemple, un kick (grosse caisse) typique a une fréquence 

In [157]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['chunk'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

**Results for chunks_240_20:**

In [133]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.980
PARTLY_RELEVANT    0.015
NON_RELEVANT       0.005
Name: proportion, dtype: float64

In [134]:
df_eval.to_csv('../data/rag-eval-gpt-4o-mini-chunks240.csv', index=False)

In [138]:
df_eval[(df_eval.relevance == 'PARTLY_RELEVANT') | (df_eval.relevance == 'NON_RELEVANT')]

,answer,id,question,relevance,explanation
31,Pour éviter les pièges du make-up gain lors de...,584d0437-3,Quels conseils professionnels peut on suivre p...,NON_RELEVANT,The generated answer discusses techniques rela...
116,Pour analyser les modifications entre les deux...,584d0437-2,Quelles modifications ai-je apportées entre le...,PARTLY_RELEVANT,The generated answer discusses various paramet...
139,"Dans Ableton, l'effet utilisé pour donner un a...",80af6c63-14,Quel effet est utilisé dans Ableton pour donne...,PARTLY_RELEVANT,The generated answer provides some relevant in...
149,Pour effectuer un test A/B dans ta session de ...,af597c08-1,Comment effectuer un test A/B dans ma session ...,PARTLY_RELEVANT,The generated answer provides steps related to...


In [171]:
df240 = pd.read_csv('../data/rag-eval-gpt-4o-mini-chunks240.csv')
df240.iloc[31]['explanation']

'The generated answer discusses techniques related to audio production and the use of compressors, which are not relevant to the question about professional advice to avoid pitfalls in makeup gain, likely related to cosmetics or makeup techniques. The content does not address the context of makeup applications or any professional tips relevant to that area.'

**Results for chunks_300_50:**

In [158]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.985
PARTLY_RELEVANT    0.015
Name: proportion, dtype: float64

In [159]:
df_eval.to_csv('../data/rag-eval-gpt-4o-mini-chunks300.csv', index=False)